## Web Project

In [21]:
from dotenv import load_dotenv
import os
import json
import requests
import pandas as pd

In [37]:
#Primera prueba API (allPrices)
source = "https://api.binance.com/api/v1/ticker/allPrices"
exchangeInfo = requests.get(source)
json_exchangeInfo = exchangeInfo.json()
sym = list(map(lambda x: {"symbol":x["symbol"],"price":x["price"]}, json_exchangeInfo))
crypto = list({v["symbol"]:v for v in sym}.values())
#print(sym)
crypto_all = pd.DataFrame(sym)
print(crypto_all)

             price      symbol
0       0.03256200      ETHBTC
1       0.01219400      LTCBTC
2       0.00240550      BNBBTC
3       0.00208700      NEOBTC
4       0.01634200     QTUMETH
5       0.02320000      EOSETH
6       0.00015881      SNTETH
7       0.00434000      BNTETH
8       0.07908100      BCCBTC
9       0.00056300      GASBTC
10      0.07387500      BNBETH
11   3663.97000000     BTCUSDT
12    119.31000000     ETHUSDT
13      0.00041400      HSRBTC
14      0.00106740      OAXETH
15      0.00009451      DNTETH
16      0.01780100      MCOETH
17      0.00166300      ICNETH
18      0.00058300      MCOBTC
19      0.00027760      WTCBTC
20      0.00848500      WTCETH
21      0.00001485      LRCBTC
22      0.00045622      LRCETH
23      0.00053400     QTUMBTC
24      0.00000391     YOYOBTC
25      0.00031400      OMGBTC
26      0.00963900      OMGETH
27      0.00006757      ZRXBTC
28      0.00207200      ZRXETH
29      0.00022010    STRATBTC
..             ...         ...
428     

In [40]:
source = "https://api.binance.com/api/v1/ticker/bookTicker"
exchangeInfo = requests.get(source)
json_exchangeInfo = exchangeInfo.json()
#print(json_exchangeInfo)

In [41]:
sym = list(map(lambda x: {"symbol":x["symbol"],"askPrice":x["askPrice"]}, json_exchangeInfo))
#print(sym)
crypto_book = pd.DataFrame(sym)
print(crypto_book)

          askPrice      symbol
0       0.03256500      ETHBTC
1       0.01220300      LTCBTC
2       0.00240550      BNBBTC
3       0.00208800      NEOBTC
4       0.01635200     QTUMETH
5       0.02323500      EOSETH
6       0.00015881      SNTETH
7       0.00433900      BNTETH
8       0.00000000      BCCBTC
9       0.00056600      GASBTC
10      0.07387500      BNBETH
11   3664.00000000     BTCUSDT
12    119.35000000     ETHUSDT
13      0.00000000      HSRBTC
14      0.00106700      OAXETH
15      0.00009459      DNTETH
16      0.01788200      MCOETH
17      0.00000000      ICNETH
18      0.00058400      MCOBTC
19      0.00027730      WTCBTC
20      0.00851200      WTCETH
21      0.00001486      LRCBTC
22      0.00045622      LRCETH
23      0.00053300     QTUMBTC
24      0.00000391     YOYOBTC
25      0.00031400      OMGBTC
26      0.00966400      OMGETH
27      0.00006762      ZRXBTC
28      0.00207799      ZRXETH
29      0.00022020    STRATBTC
..             ...         ...
428     